Get distance to nearest TSS for each locus

In [81]:
%pylab inline
import matplotlib

Populating the interactive namespace from numpy and matplotlib


In [82]:
annotations_file_name = '/storage/BonnieH/selection_project/annotations/ccds.txt'
#sistr_file_name = '/storage/BonnieH/selection_project/per_locus/SISTR_results/SSC_scores.txt'
sistr_file_name = '/storage/BonnieH/selection_project/1000genomes/euro_allele_freqs/allele_freqs_filt_w_total_thresh.txt'

In [83]:
def getNearestDist(s_ABC_round, s_list_available):
    min_dist = 100000000000000000000000000000000
    nearest_s = -2
    for elem in s_list_available:
        dist = abs(s_ABC_round - elem)
        if dist < min_dist:
            min_dist = dist
            nearest_s = elem
    return nearest_s

In [84]:
def main():
    # Get all TSS

    results = open('/storage/BonnieH/selection_project/1000genomes/euro_allele_freqs/allele_freqs_w_dist_annot.txt', 'w')
    
    minus_tss = {}
    plus_tss = {}
    chrom_numbers = list(range(1,23))
    chrom_list = []

    for chrom in chrom_numbers:
        minus_tss['chr'+str(chrom)] = []
        plus_tss['chr'+str(chrom)] = []
        chrom_list.append('chr' + str(chrom))

    annotations_file = open(annotations_file_name, 'r')
    header = annotations_file.readline().strip().split('\t')
    
    # Get information from line
    for line in annotations_file:
        info = line.strip().split('\t')
        chrom = info[0]
        start = int(info[1])
        end = int(info[2])
        strand = info[5]
        if chrom in chrom_list:
            if strand == '+':
                if start not in plus_tss[chrom]:
                    plus_tss[chrom].append(start)

            if strand == '-':
                if end not in minus_tss[chrom]:
                    minus_tss[chrom].append(end)

    annotations_file.close()
    print(len(minus_tss))
    print(len(plus_tss))
    #for chrom in chrom_numbers:
        #print(len(plus_tss['chr'+str(chrom)]))
    #print(minus_tss[1])
    
    sistr_file = open(sistr_file_name, 'r')
    #header = sistr_file.readline().strip().split('\t')
    
    bin_incr =1000
    num_bins= int(20000/bin_incr)
   
    start_bin_list = []
   
    # Get information from line
    for line in sistr_file:
        
        info = line.strip().split('\t')
        chrom = int(info[0])
        start = int(info[1])
        end = int(info[2])
        middle = int((start+end)/2)
        period = int(info[5])
        write = False
        motif = info[6]
        if period == 2 or period == 3 or period == 4: # and opt_allele == 5: #motif == motif_to_use: # total > 5214
            line= line.rstrip("\n")
            results.write(line)
           
            plus_list = plus_tss['chr'+str(chrom)]
            minus_list = minus_tss['chr'+str(chrom)]
            plus_closest = getNearestDist(middle, plus_list)
            minus_closest = getNearestDist(middle, minus_list)
            
            if abs(middle-plus_closest) < abs(minus_closest-middle):
                distance = middle-plus_closest

            else:
                distance = minus_closest-middle
            
            if distance < 10000 and distance > -10000:
                
                start_bin = -10000
                count = 0
                
                for k in range(0, num_bins):
                    if distance > start_bin and distance < start_bin + bin_incr:
                        
                        results.write('\t' + str(start_bin) + '\n')
                        write = True
                        count = count + 1
                        start_bin_list.append(start_bin)
                        break
                    start_bin = start_bin + bin_incr
            if write == False:
                results.write('\t' + str(-1) + '\n')
            
            
    sistr_file.close()
    print(start_bin_list.count(-10000))
    print(start_bin_list.count(-9000))
    print(start_bin_list.count(-8000))
    print(start_bin_list.count(-7000))
    print(start_bin_list.count(-6000))
    print(start_bin_list.count(-5000))
    results.close()

In [85]:
%%time
if __name__ == '__main__':
    main()

22
22
536
519
542
548
622
641
CPU times: user 16.4 s, sys: 60 ms, total: 16.5 s
Wall time: 16.5 s
